In [ ]:
import numpy as np
import altair as alt
import pandas as pd
from vega_datasets import data


In [ ]:
#!pip install vega-datasets

In [ ]:
# Look in the Interaction and Map notebooks on the CSE 512 course website for help coding!

usa = data.us_10m.url
usa

In [ ]:
alt.Chart(alt.topo_feature(usa, 'counties')).mark_geoshape(
    fill='#ddd', stroke='#fff', strokeWidth=1
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

In [ ]:
income = pd.read_csv('CAINC1__ALL_AREAS_1969_2022.csv', encoding='latin-1')#,skiprows=4)
income.head(10)

In [ ]:
income2 = income.melt(id_vars=['GeoFIPS','GeoName','Region','TableName','LineCode','IndustryClassification','Description','Unit'], var_name="year", value_name="Value")
income2 = income2.dropna()
income2.columns = income2.columns.str.replace('GeoFIPS', 'county_fips')
income2['county_fips'] = income2['county_fips'].str.replace('"','')
income2['county_fips'] = income2['county_fips'].str.strip()
income2['year'] = income2['year'].astype(int)
#income2['county_fips'].dtype
income2 = income2[income2['Description'] == 'Per capita personal income (dollars) 2/']
income2.head()

In [ ]:
pres2020 = pd.read_csv('countypres_2000-2020.csv')
pres2020 = pres2020.dropna()
pres2020['county_fips'] = pres2020['county_fips'].astype(int)
pres2020['county_fips'] = '0' + pres2020['county_fips'].astype(str)
pres2020['voteshare'] = pres2020['candidatevotes'] / pres2020['totalvotes']
pres2020.head()

In [ ]:
comb = pd.merge(income2, pres2020[['county_fips','party','year','voteshare','candidatevotes']], on=['county_fips','year'])
#comb = comb.head(100)

In [ ]:
alt.data_transformers.disable_max_rows()

slider = alt.binding_range(min=2000, max=2022, step=4, name='Current Year: ')

selector = alt.selection_point(
    name="SelectorName",
    fields=['year'],
    bind=slider,
    value=[{'time': 2000}]
)

graph = alt.Chart(comb).mark_point(size=50,filled=True).encode(
    alt.X('Value:Q').title(['Income Per Capita']),#.sort(['Jan']),
    alt.Y('voteshare:Q').title(['voteshare']),
    alt.Tooltip('GeoName'),
    alt.Size('candidatevotes'),
    alt.Color('party')
).add_params(selector).transform_filter(
    selector
)

graph

In [ ]:
eurovote = pd.read_csv('eu_ned_national.csv')
eurovote.columns = eurovote.columns.str.replace('nuts2016', 'REF_AREA')
eurovote.columns = eurovote.columns.str.replace('year', 'TIME_PERIOD')
eurovote['voteshare'] = eurovote['partyvote'] / eurovote['validvote']
eurovote.head()
#len(eurovote)

In [ ]:
eurogdp = pd.read_csv('OECDGDPcodes.csv')
eurogdp = eurogdp.loc[:,['TERRITORIAL_LEVEL','TIME_PERIOD','OBS_VALUE','COUNTRY','REF_AREA']]
#eurogdp = eurogdp[eurogdp['REF_AREA'] == 'AT111']
eurogdp.head()

In [ ]:
partycode = pd.read_csv('V-Dem-CPD-Party-V2.csv')
#partycode = partycode.loc[:,['year','pf_party_id','v2xpa_antiplural','v2paanteli']]
partycode = partycode.loc[:,:]
partycode.columns = partycode.columns.str.replace('year', 'TIME_PERIOD')
partycode.columns = partycode.columns.str.replace('pf_party_id', 'partyfacts_id')
partycode.head()
#partycode['TIME_PERIOD'].info()
#artycode['pf_party_id'].info()

In [ ]:
eurocombo = pd.merge(eurovote, eurogdp[['REF_AREA','TIME_PERIOD', 'OBS_VALUE']], on=['REF_AREA','TIME_PERIOD'], how='left')

eurogdp2 = pd.read_csv('OECDGDPcodes_some_convertnuts2016.csv')
eurogdp2 = eurogdp2.loc[:,['TERRITORIAL_LEVEL','TIME_PERIOD','OBS_VALUE','COUNTRY','REF_AREA']]
eurocombo = pd.merge(eurocombo, eurogdp2[['REF_AREA','TIME_PERIOD', 'OBS_VALUE']], on=['REF_AREA','TIME_PERIOD'], how='left')
eurocombo.loc[eurocombo['OBS_VALUE_y'].notna(), 'OBS_VALUE_x'] = eurocombo['OBS_VALUE_y']
eurocombo = eurocombo.dropna(subset=['OBS_VALUE_x','OBS_VALUE_y'])

eurocombo.columns = eurocombo.columns.str.replace('OBS_VALUE', 'GDP per capita (PPP, constant prices 2015)')
eurocombo = pd.merge(eurocombo, partycode, on=['TIME_PERIOD','partyfacts_id'])
eurocombo.head()

In [ ]:
#jam = pd.read_csv('OECDGDPcodes.csv')
jam = eurovote[eurovote['country_code'] == "NO"]
#jam = eurogdp[eurogdp['COUNTRY'] == "NOR"]
# jam = eurogdp[eurogdp['REF_AREA'] != 'NO']
#jam = eurocombo[eurocombo['country_code'] == 'NO']
#jam = jam[jam['REF_AREA'] != 'NO071']
#jam['TIME_PERIOD'].unique()
jam['REF_AREA'].unique()
#jam.head(120)

In [ ]:
eurocombo['country'].unique()

In [ ]:
alt.data_transformers.disable_max_rows()

# Make radio button less cramped by adding a space after each label
# The spacing will only show up in your IDE, not on this doc page
options = list(eurocombo['country'].unique())
labels = [option + ' ' for option in options]

input_dropdown = alt.binding_select(
    # Add the empty selection which shows all when clicked
    options=options + [None],
    labels=labels + ['All'],
    name='Region: '
)
countryselection = alt.selection_point(
    fields=['country'],
    bind=input_dropdown,
)

slider2 = alt.binding_range(min=eurocombo['TIME_PERIOD'].min(), 
                            max=eurocombo['TIME_PERIOD'].max(), step=1, name='Current Year: ')

yearselector = alt.selection_point(
    name="SelectorName",
    fields=['TIME_PERIOD'],
    bind=slider2,
    value=[{'TIME_PERIOD': 2000}]
)

partyselect = alt.selection_point(fields=['party_abbreviation'], bind='legend')

base = alt.Chart(eurocombo).mark_point().encode(
    x='GDP per capita (PPP, constant prices 2015)_x:Q',
    y='voteshare:Q',
    # We need to set a constant domain to preserve the colors
    # when only one region is shown at a time
    tooltip=['country','regionname','party_abbreviation'],
    size=alt.Size('partyvote'),
    #opacity=alt.condition(partyselect, alt.value(1), alt.value(0.2)),
    #.scale(domain=options),
).add_params(
    countryselection
).transform_filter(
    countryselection
).add_params(
    yearselector
).transform_filter(yearselector).add_params(partyselect)

main = base.encode(
    color=alt.Color('party_abbreviation:N'),
    opacity=alt.condition(partyselect, alt.value(1), alt.value(0.2))
)

regres = base.transform_filter(
    partyselect
).transform_regression('GDP per capita (PPP, constant prices 2015)_x', 
                                   'voteshare').mark_line(color='black',opacity=1)

# .encode(
#                                        opacity=alt.condition(partyselect, alt.value(1), alt.value(0)
#                                    ))
main+regres

In [ ]:
import altair as alt
from vega_datasets import data

cars = data.cars()

selection_origin = alt.selection_multi(fields=['Origin'])
selection_cylinders = alt.selection_multi(fields=['Cylinders'])

color_origin = alt.condition(
    selection_origin,
    alt.Color('Origin:N', legend=None),
    alt.value('lightgray')
)

color_cylinders = alt.condition(
    selection_cylinders,
    alt.Color('Cylinders:O', legend=None),
    alt.value('lightgray')
)

scatter = alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=color_origin,
    tooltip='Name:N'
).add_selection(
    selection_origin
).add_selection(
    selection_cylinders
)


legend_origin = alt.Chart(cars).mark_rect().encode(
    y=alt.Y('Origin:N', axis=alt.Axis(orient='right')),
    color=color_origin
).add_selection(
    selection_origin
)

legend_cylinders = alt.Chart(cars).mark_rect().encode(
    y=alt.Y('Cylinders:O', axis=alt.Axis(orient='right')),
    color=color_cylinders
).add_selection(
    selection_cylinders
)

scatter | legend_origin | legend_cylinders


In [ ]:
import pandas as pd
import altair as alt

# Create dataframe
data = [[7, 10, 'Alex', 'Smith'],
        [12, 20, 'Bob', 'Jones'],
        [10, 30, 'Clive', 'Smith'],
        [42, 40,  'Alex', 'Johnson']]
df = pd.DataFrame(data,columns=['Favorite number', 'Age', 'First name', 'Last name'])

# Create selections
selection_first_name = alt.selection_multi(fields=['First name'], empty='none')
selection_last_name = alt.selection_multi(fields=['Last name'], empty='none')

# Create interactive scatter plot
scatter = alt.Chart(df).mark_point(size=100).encode(
    x='Favorite number:Q',
    y='Age:Q',
    color=alt.condition(selection_first_name & selection_last_name,
                        alt.Color('First name:N', legend=None),
                        alt.value('lightgray') ),
    shape=alt.Shape('Last name:N', legend=None),
    tooltip=['First name', 'Last name']
).add_selection(
    alt.selection_interval(bind='scales')
)

# Create interactive model name legend
legend_first_name = alt.Chart(df).mark_point(size=100).encode(
    y=alt.Y('First name:N', axis=alt.Axis(orient='right')),
    color=alt.condition(selection_first_name,
                        alt.Color('First name:N', legend=None),
                        alt.value('lightgray') ),
).add_selection(
    selection_first_name

)

# Create interactive model name legend
legend_last_name = alt.Chart(df).mark_point(size=100).encode(
    y=alt.Y('Last name:N', axis=alt.Axis(orient='right')),
    shape=alt.Shape('Last name:N', legend=None),
    color=alt.condition(selection_last_name,
                        alt.value('black'),
                        alt.value('lightgray') ),
).add_selection(
    selection_last_name
)

# Combine plotting elements
chart = scatter | legend_first_name | legend_last_name
chart

In [ ]:
print(eurocombo.shape)

In [ ]:
unemp = {
    'idu':[4005,4005,4017,4017],
    'rate':[0.4,3,0.3,0.4],
    'year':[2000,2001,2000,2001]

}
unemp = pd.DataFrame(unemp)

from vega_datasets import data
counties = alt.topo_feature(data.us_10m.url, 'counties')
unemp_data = data.unemployment.url
print(unemp_data)
alt.Chart(unemp).transform_lookup(
    lookup='idu',
    from_=alt.LookupData(counties, 'id'), as_='geom'#, ['rate','year'])
# ).transform_aggregate(
#     Gross='sum(rate)',
#     groupby=['id','geom']
).transform_aggregate(
    Gross='sum(rate)',
    groupby=['idu','geom']
).transform_calculate(
    Gross2 = 'datum.Gross',
    geometry ='datum.geom.geometry',
    type= 'datum.geom.type'
).mark_geoshape().encode(
    color='Gross:Q',
    tooltip=['id:N','Gross2:Q']
).project(
    type='identity',reflectY=True
).properties(
    #projection={'type': 'identity'},
    width=500, height=300
)


# .transform_filter(
#     'datum.year == 2001'
# ).transform_aggregate(
#     Gross='mean(rate)',
#     groupby=['id']
# )

In [ ]:
import altair as alt
import pandas as pd

# Assuming df is your DataFrame
df = pd.DataFrame({
    'country': ['A', 'B', 'C', 'D'],
    'gdp': [1, 2, 3, 4],
    'immi': [2, 6, 11, 15],
    'lgb': [1.5, 2.5, 3.5, 4.5]
})

# Define the dropdown selections
selection_field = alt.binding_select(options=['immi', 'lgb'])
selection_range = alt.binding_select(options=[(0, 5), (5, 10), (10, 15), (1, 2), (2, 3), (3, 4)])

# Create two input elements and bind them to the selections
input_dropdown_field = alt.binding_select(options=['immi', 'lgb'])
input_dropdown_range = alt.binding_select(options=[(0, 5), (5, 10), (10, 15), (1, 2), (2, 3), (3, 4)])

selection_field = alt.selection_single(fields=['field'], bind=input_dropdown_field, name='Select field',
                                        #init={'field': 'immi'}
                                        )
selection_range = alt.selection_single(fields=['lower', 'upper'], bind=input_dropdown_range, name='Select range', 
                                       #init={'lower': 0, 'upper': 5}
                                       )

# Create a scatter plot
scatter = alt.Chart(df).mark_circle().encode(
    x='gdp:Q',
    y='country:N',
    tooltip=['country', 'gdp', 'immi', 'lgb']
).add_selection(
    selection_field,
    selection_range
).transform_filter(
    alt.FieldRangePredicate(field='immi', range=[0, 5])
).transform_filter(
    alt.FieldRangePredicate(field='lgb', range=[1, 2])
)

scatter


In [ ]:
import altair as alt
import pandas as pd

# Assuming df is your DataFrame
df = pd.DataFrame({
    'country': ['A', 'B', 'C', 'D'],
    'gdp': [1, 2, 3, 4],
    'immi': [2, 6, 11, 15],
    'lgb': [1.5, 2.5, 3.5, 4.5]
})

# Define a new DataFrame 'df_extended' that categorizes the 'immi' and 'lgb' values into the specified ranges
df_immi = df.loc[df['immi'].between(0, 5)].copy()
df_immi['range'] = 'immi 0-5'
df_extended = df_immi

ranges = [('immi', 5, 10, 'immi 5-10'), ('immi', 10, 15, 'immi 10-15'), 
          ('lgb', 1, 2, 'lgb 1-2'), ('lgb', 2, 3, 'lgb 2-3'), ('lgb', 3, 4, 'lgb 3-4')]

for column, lower, upper, range_name in ranges:
    df_temp = df.loc[df[column].between(lower, upper)].copy()
    df_temp['range'] = range_name
    df_extended = pd.concat([df_extended, df_temp])

# Define the dropdown selection
selection = alt.selection_single(
    name='Select',
    fields=['range'],
    #init={'range': 'immi 0-5'},
    bind=alt.binding_select(options=[
        'immi 0-5', 'immi 5-10', 'immi 10-15', 
        'lgb 1-2', 'lgb 2-3', 'lgb 3-4'
    ])
)

# Create a scatter plot
scatter = alt.Chart(df_extended).mark_circle().encode(
    x='gdp:Q',
    y='country:N',
    tooltip=['country', 'gdp', 'immi', 'lgb']
).transform_filter(
    selection
).add_selection(
    selection
)

#scatter
df_extended


In [ ]:
for i in np.arange(0,1.5,0.5):
    print(i)

In [ ]:
partycode.columns

In [ ]:
import pandas as pd
import numpy as np

# Original DataFrame
df = pd.DataFrame({
    'place': ['france', 'france', 'france', 'germany', 'germany', 'france', 'france', 'france', 'germany', 'germany'],
    'party': ['spd', 'spd', 'spd', 'cdu', 'cdu', 'ricky', 'micky', 'vicky', 'dicky', 'picky'],
    'amount': [200, 300, 250, 200, 300, 900, 800, 700, 600, 500],
    'year': [2004, 2010, 2015, 2007, 2010, 2004, 2010, 2015, 2007, 2010]
})

# Define the start and end years for each place
start_end_years = {
    'france': [2004, 2016],
    'germany': [2007, 2016]
}

# Initialize an empty DataFrame for the result
new_df = pd.DataFrame()

# Loop over each place
for place, (start_year, end_year) in start_end_years.items():
    # Create a DataFrame for this place with the desired years
    df_place = pd.DataFrame({'year': list(range(start_year, end_year)), 'place': place})
    
    # Get the unique parties for this place
    parties = df[df['place'] == place]['party'].unique()
    
    # Create a DataFrame with all combinations of year and party
    df_place = df_place.assign(key=1)
    df_parties = pd.DataFrame({'party': parties, 'key': 1})
    df_place = pd.merge(df_place, df_parties, on='key').drop('key', axis=1)
    
    # Merge with the original DataFrame
    df_place = pd.merge(df_place, df[df['place'] == place], on=['year', 'place', 'party'], how='left')
    
    # Forward fill the missing values separately for each party
    df_place['amount'] = df_place.groupby('party')['amount'].ffill()
    
    # Append to the result DataFrame
    new_df = pd.concat([new_df, df_place])

# Replace NaN values with a specific value (e.g., 0 or np.nan)
new_df['amount'].fillna(0, inplace=True)

print(new_df)


In [ ]:
euromap2.head()
'country', 'country_code','nutslevel','REF_AREA','regionname','type','TIME_PERIOD'

In [ ]:
topojson_data_nuts3 = 'NUTS_RG_03M_2016_3035_LEVL_3.json'
nuts3geo = alt.topo_feature(topojson_data_nuts3, 'NUTS_RG_03M_2016_3035')

euromap2 = eurovote.copy()
notinclude = ['FRY10','FRY20','FRY30','FRY40','FRY50', 'PT200', 'PT300',
              'ES707', 'ES703', 'ES706', 'ES709', 'ES705', 'ES704', 'ES708']
euromap2 = euromap2[~euromap2['REF_AREA'].isin(notinclude)]

euromap2 = pd.merge(euromap2, 
                    partycode[['TIME_PERIOD','partyfacts_id','v2xpa_antiplural']], 
                    on=['TIME_PERIOD','partyfacts_id'])

metricintervals = {'v2xpa_antiplural':1,
                   'v2xpa_popul':1,
                    'v2pariglef':6,
                   'v2pawelf':5}

new_df = pd.DataFrame({'year': list(range(2004, 2016))})

# relevantcols = []
# for colname in list(euromap2.columns):
#     if('v2' in colname):
#         maxval = 0
#         step = 1
#         if(colname in metricintervals.keys()):
#             maxval = metricintervals[colname]
#             if(maxval == 1):
#                 step = 0.2
#         else:
#             maxval = 4
        
#         for i in range(0,maxval-step,step):
#             #euromap2.append
#             euromap2[colname + ' '+ str(i) + ' to ' + str(i+step)] = (euromap2[colname] > i) & (euromap2[colname] < i + step)
        
# euromap2['antipluralcat'] = '0 to 0.5'
# euromap2.loc[euromap2['v2xpa_antiplural'] > 0.5, 'antipluralcat'] = '0.5 to 1'

####################################################

# firstcol = list(euromap2.columns)[0]
# df_immi = euromap2.loc[euromap2['v2pawelf'].between(0, 5)].copy()
# df_immi['v2pawelf'] = 'immi 0-5'
# df_extended = df_immi

# ranges = []
# names = ['immi 0-5']
# for colname in list(euromap2.columns)[1:]:
#     #if('v2' in colname):
#     if(colname in metricintervals.keys()):
#         maxval = 0
#         step = 1

#         if(colname in metricintervals.keys()):
#             maxval = metricintervals[colname]
#             if(maxval == 1):
#                 step = 0.2
#         else:
#             maxval = 4
        
#         for i in np.arange(0,maxval-step,step):
#             ranges.append((colname,i,i+step,colname + ' '+ str(i) + ' to ' + str(i+step)))
#             names.append(colname + ' '+ str(i) + ' to ' + str(i+step))
#             #euromap2[colname + ' '+ str(i) + ' to ' + str(i+step)] = (euromap2[colname] > i) & (euromap2[colname] < i + step)
# print(ranges)
# # ranges = [('immi', 5, 10, 'immi 5-10'), ('immi', 10, 15, 'immi 10-15'), 
# #           ('lgb', 1, 2, 'lgb 1-2'), ('lgb', 2, 3, 'lgb 2-3'), ('lgb', 3, 4, 'lgb 3-4')]

# for column, lower, upper, range_name in ranges:
#     df_temp = euromap2.loc[euromap2[column].between(lower, upper)].copy()
#     df_temp['range'] = range_name
#     df_extended = pd.concat([df_extended, df_temp])

# # Define the dropdown selection
# selectionspecial = alt.selection_single(
#     name='Select',
#     fields=['range'],
#     #init={'range': 'immi 0-5'},
#     bind=alt.binding_select(options=names)
# )
#############################################

# Create a dropdown selection
euromap2['antipluralcat'] = '0 to 0.5'
euromap2.loc[euromap2['v2xpa_antiplural'] > 0.5, 'antipluralcat'] = '0.5 to 1'
selectiondrop2 = alt.selection_point(fields=['antipluralcat'], 
                                     bind=alt.binding_select(options=['0 to 0.5', '0.5 to 1']),
                                     value=[{'antipluralcat': '0 to 0.5'}])

ref_area_selection = alt.selection_single(fields=['REF_AREA'], bind=alt.binding_select(options=list(euromap2['REF_AREA'].unique())))

slider_more_time = alt.binding_range(min=euromap2['TIME_PERIOD'].min(), 
                            max=euromap2['TIME_PERIOD'].max(), step=1, name='Current Year: ')

yearselector = alt.selection_point(
    name="SelectorName",
    fields=['TIME_PERIOD'],
    bind=slider_more_time,
    value=[{'TIME_PERIOD': 2000}]
)

topojson_data_nuts3 = 'NUTS_RG_03M_2016_3035_LEVL_3.json'
base = alt.Chart(alt.topo_feature(topojson_data_nuts3, 'NUTS_RG_03M_2016_3035')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25, fill='lightgray'
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=euromap2, key='REF_AREA', fields=['totalvote'])
).transform_filter(
    {'not':alt.FieldOneOfPredicate(field='id', oneOf=notinclude)}
).encode().project(
    type='identity',reflectY=True
).properties(
    width=900,
    height=500
)

lets = alt.Chart(euromap2).transform_lookup(
    lookup='REF_AREA', from_=alt.LookupData(data=nuts3geo, key='id'), as_='geom'#, fields=['totalvote','REF_AREA','voteshare','antipluralcat','v2xpa_antiplural'])
).transform_filter(
    'datum.geom !== null'  # Filter out records without a match in source2
# ).add_params(
#     yearselector
# ).transform_filter(
#     yearselector
).add_params(selectiondrop2
).transform_filter(
    selectiondrop2
).transform_aggregate(
    Gross='sum(voteshare)',
    groupby=['REF_AREA','geom','TIME_PERIOD']
).transform_calculate(
    Gross2 = 'datum.Gross',
    geometry ='datum.geom.geometry',
    type= 'datum.geom.type'
).mark_geoshape(
    #stroke='#aaa', strokeWidth=0.25
).encode(
    alt.Color('Gross:Q'),
    tooltip = [alt.Tooltip('Gross:Q'),#, format='.0%'),
               alt.Tooltip('REF_AREA:N'),
               alt.Tooltip('v2xpa_antiplural:Q')]
).project(
    type='identity',reflectY=True

).properties(
    width=900,
    height=500
)

#lets.mark_geoshape(stroke='#aaa', strokeWidth=0.25, fill='lightgray')
base.mark_geoshape(fill='lightgray') + lets.add_params(yearselector).transform_filter(yearselector)
# .transform_filter(
#     selectionspecial
# ).add_selection(
#     selectionspecial
# )
# .add_params(selectiondrop2
# ).transform_filter(
#     selectiondrop2
# ) 
#.mark_geoshape(stroke='#aaa', strokeWidth=0.25, fill='lightgray')

In [ ]:
'ES707', 'ES703', 'ES706', 'ES709', 'ES705', 'ES704', 'ES708'

In [ ]:
eurotest = eurovote
yesinclude = ['FRB01','ES212']
eurotest = eurovote[eurovote['REF_AREA'].isin(yesinclude)]
eurotest = pd.merge(eurotest, partycode, on=['TIME_PERIOD','partyfacts_id'])
#eurotest.head()

slidertest = alt.binding_range(min=eurotest['TIME_PERIOD'].min(), 
                            max=eurotest['TIME_PERIOD'].max(), step=1, name='Current Year: ')

yearselectortest = alt.selection_point(
    name="SelectorName",
    fields=['TIME_PERIOD'],
    bind=slider2,
    value=[{'TIME_PERIOD': 2000}]
)

topojson_data_nuts3 = 'NUTS_RG_03M_2016_3035_LEVL_3.json'
testmap = alt.Chart(alt.topo_feature(topojson_data_nuts3, 'NUTS_RG_03M_2016_3035')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=eurotest, key='REF_AREA', 
    fields=['totalvote','REF_AREA','voteshare','antipluralcat','v2xpa_antiplural','partyfacts_id','TIME_PERIOD'])
# ).transform_aggregate(
#     Gross='sum(totalvote)',
#     groupby=['REF_AREA']
).encode(
    alt.Color('totalvote:Q'),
    tooltip = [alt.Tooltip('totalvote:Q'),#, format='.0%'),
               alt.Tooltip('REF_AREA:N'),
               alt.Tooltip('v2xpa_antiplural:Q'),
               alt.Tooltip('partyfacts_id:Q'),
               alt.Tooltip('TIME_PERIOD:O'),]
).project(
    type='identity',reflectY=True
).add_params(
    yearselector
).transform_filter(yearselector
).properties(
    width=900,
    height=500
)
testmap

In [ ]:
alt.data_transformers.disable_max_rows()

notinclude = ['FRY10','FRY20','FRY30','FRY40','FRY50', 'PT200', 'PT300']
euromap = eurovote[~eurovote['REF_AREA'].isin(notinclude)]

euromap = pd.merge(euromap, partycode[['TIME_PERIOD','partyfacts_id','v2xpa_antiplural']], on=['TIME_PERIOD','partyfacts_id'])
euromap['antipluralcat'] = '0 to 0.5'
euromap.loc[euromap['v2xpa_antiplural'] > 0.5, 'antipluralcat'] = '0.5 to 1'

# Create a dropdown selection
selectiondrop = alt.selection_point(fields=['antipluralcat'], bind=alt.binding_select(options=['0 to 0.5', '0.5 to 1']))

#include = ['NO073','NO022','NO071','NO072','NO061','NO062','NO060']
#euromap = eurovote[eurovote['REF_AREA'].isin(include)]

# topojson_data_nuts3 = 'NUTS_RG_03M_2016_3035_LEVL_3.json'
# nuts3map = alt.Chart(alt.topo_feature(topojson_data_nuts3, 'NUTS_RG_03M_2016_3035')).mark_geoshape(
#     stroke='#aaa', strokeWidth=0.25
# ).transform_lookup(
#     lookup='id', from_=alt.LookupData(data=euromap, key='REF_AREA', fields=['totalvote','REF_AREA','voteshare','antipluralcat','v2xpa_antiplural'])
# ).encode(
#     alt.Color('totalvote:Q'),
#     tooltip = [alt.Tooltip('totalvote:Q'),#, format='.0%'),
#                alt.Tooltip('REF_AREA:N'),
#                alt.Tooltip('v2xpa_antiplural:Q')]
# ).project(
#     type='identity',reflectY=True
# ).add_params(selectiondrop
# ).transform_filter(
#     selectiondrop
# ).properties(
#     width=900,
#     height=500
# )
# # .configure_view(
# #     stroke=None
# # )

# topojson_data_nuts2 = 'NUTS_RG_03M_2016_3035_LEVL_2.json'
# nuts2map = alt.Chart(alt.topo_feature(topojson_data_nuts2, 'NUTS_RG_03M_2016_3035')).mark_geoshape(
#     stroke='#aaa', strokeWidth=0.25, fill='lightgray'
# ).transform_lookup(
#     lookup='id', from_=alt.LookupData(data=euromap, key='REF_AREA', fields=['totalvote','REF_AREA'])
# ).encode(
#     alt.Color('totalvote:Q'),
#     tooltip = [alt.Tooltip('totalvote:Q'),#, format='.0%'),
#                alt.Tooltip('REF_AREA:N')]
# ).project(
#     type='identity',reflectY=True
# ).properties(
#     width=900,
#     height=500
# )

# topojson_data_2021 = 'NUTS_RG_03M_2021_3035_LEVL_2.json'
# nuts2map2021 = alt.Chart(alt.topo_feature(topojson_data_nuts2, 'NUTS_RG_03M_2021_3035')).mark_geoshape(
#     stroke='#aaa', strokeWidth=0.25
# ).transform_lookup(
#     lookup='id', from_=alt.LookupData(data=euromap, key='REF_AREA', fields=['totalvote','REF_AREA']
# )).encode(
#     alt.Color('totalvote:Q'),
#     tooltip = [alt.Tooltip('totalvote:Q'),#, format='.0%'),
#                alt.Tooltip('REF_AREA:N')]
# ).project(
#     type='identity',reflectY=True
# ).properties(
#     width=900,
#     height=500
# )


votemapborder = alt.Chart(alt.topo_feature('NUTS_RG_03M_2021_3035_LEVL_0.json', 'NUTS_RG_03M_2021_3035')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.5
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=euromap, key='REF_AREA', fields=['totalvote','REF_AREA'])
).encode().project(
    type='identity',reflectY=True
).properties(
    width=900,
    height=500
)
# .configure_view(
#     stroke=None
# )
votemapborder
#nuts3map + nuts2map
#.mark_geoshape(stroke='#aaa', strokeWidth=0.25, fill='lightgray')+nuts2map

In [ ]:
#numeric_columns = eurovote.select_dtypes(include=[np.number]).columns.tolist()
#europarty = eurovote.groupby(numeric_columns).sum()

europarty = eurovote.groupby(['country','country_code','party_abbreviation','TIME_PERIOD','partyfacts_id']).sum(numeric_only=True).reset_index()
europarty = pd.merge(europarty, partycode, on=['TIME_PERIOD','partyfacts_id'])
europarty.head()

In [ ]:
alt.data_transformers.disable_max_rows()

nearest = alt.selection_point(nearest=True, on='click', fields=['country'], bind='legend')#, empty='none')

topojson_data = 'NUTS_RG_03M_2021_3035_LEVL_0.json'
base2 = alt.Chart(alt.topo_feature(topojson_data, 'NUTS_RG_03M_2021_3035')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=europarty, key='country_code', fields=['v2xpa_antiplural'])
).encode(
    alt.Color('v2xpa_antiplural:Q',
              #scale=alt.Scale(domain=[0, 0.3], clamp=True), 
              #legend=alt.Legend(format='%')
             ),
    alt.Tooltip('v2xpa_antiplural:Q'),#, format='.0%')
   # opacity=alt.condition(nearest, alt.value(1), alt.value(0.2)),

).project(
    type='identity',reflectY=True
).properties(
    width=900,
    height=500
).add_params(
    nearest)

input_dropdown = alt.binding_select(
    # Add the empty selection which shows all when clicked
    options=options + [None],
    labels=labels + ['All'],
    name='Region: '
)
countryselection = alt.selection_point(
    fields=['country'],
    bind=input_dropdown,
)

indicators = ['v2xpa_antiplural','v2paanteli']
policy_dropdown = alt.binding_select(
    # Add the empty selection which shows all when clicked
    #options=options + [None],
    options = indicators,
    #labels=labels + ['All'],
    name='Policy: '
)
policyselection = alt.selection_point(
    fields=['option'],
    value = 'v2xpa_antiplural',
    bind=policy_dropdown,
)
slider_more_time = alt.binding_range(min=europarty['TIME_PERIOD'].min(), 
                            max=europarty['TIME_PERIOD'].max(), step=1, name='Current Year: ')

yearselector = alt.selection_point(
    name="SelectorName",
    fields=['TIME_PERIOD'],
    bind=slider_more_time,
    value=[{'TIME_PERIOD': 2000}]
)

base3 = alt.Chart(europarty).mark_point().encode(
    x='party_abbreviation:N',
    y='value:Q',
    tooltip=['country','party_abbreviation','TIME_PERIOD','v2xpa_antiplural'],
    # We need to set a constant domain to preserve the colors
    # when only one region is shown at a time
    #egionname','party_abbreviation'],
    #size=alt.Size('partyvote'),
    #opacity=alt.condition(nearest, alt.value(1), alt.value(0.2)),
    #.scale(domain=options),
).transform_fold(
    indicators,
    as_=['option', 'value']
).add_params(
    policyselection
).transform_filter(
    policyselection
).add_params(
    countryselection
).transform_filter(
    countryselection
).add_params(
    yearselector
).transform_filter(yearselector)
#.add_params(
#    nearest).transform_filter(nearest)

#.configure_view(
 #   stroke=None
#)

base3

In [ ]:
world_data = pd.read_csv('V-Dem-CPD-Party-V2.csv')

world_data.head()

In [ ]:
print(world_data.shape)

In [ ]:
world_data = world_data[world_data['year'] > 1970]

In [ ]:
print(world_data.shape)

In [ ]:
eu_countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark',
  'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland',
  'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands',
  'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']

In [ ]:
eu_data = world_data.loc[world_data['country_name'].isin(eu_countries)]

eu_data.head()

In [ ]:
print(eu_data.shape)

In [ ]:
print(eu_data.shape)

In [ ]:
alt.data_transformers.disable_max_rows()

# Make radio button less cramped by adding a space after each label
# The spacing will only show up in your IDE, not on this doc page
options = list(eu_data['country_name'].unique())
labels = [option + ' ' for option in options]

input_dropdown = alt.binding_select(
    # Add the empty selection which shows all when clicked
    options=options,
    labels=labels,
    name='Region: '
)

countryselection = alt.selection_point(
    fields=['country_name'],
    bind=input_dropdown,
)

slider2 = alt.binding_range(min=eu_data['year'].min(), 
                            max=eu_data['year'].max(), step=1, name='Current Year: ')

yearselector = alt.selection_point(
    name="YearSelector",
    fields=['year'],
    bind=slider2,
    value=[{'year': 2000}]
)

# partyselect = alt.selection_point(fields=['party_abbreviation'], bind='legend')

base = alt.Chart(eu_data).mark_point().encode(
    x=alt.X('v2paminor:Q', scale=alt.Scale(domain=[-5, 5])),
    y=alt.Y('v2paimmig:Q', scale=alt.Scale(domain=[-5, 5])),
    
    # We need to set a constant domain to preserve the colors
    # when only one region is shown at a time
    # tooltip=['country','regionname','party_abbreviation'],
    # size=alt.Size('partyvote'),
    #opacity=alt.condition(partyselect, alt.value(1), alt.value(0.2)),
    #.scale(domain=options),
).add_params(
    countryselection
).transform_filter(
    countryselection
).add_params(
    yearselector
).transform_filter(
    yearselector
)


base.display()

In [ ]:
print(eu_data['v2panumbseat'])

In [ ]:
print(eu_data['v2patotalseat'])

In [ ]:
world_data['v2paseatpercentage'] = world_data['v2panumbseat'] / world_data['v2patotalseat']

print(world_data['v2paseatpercentage'])

In [ ]:
print(world_data.shape)
print(eu_data.shape)

In [ ]:
print(eu_data['v2paimmig'].max())

In [ ]:
alt.data_transformers.disable_max_rows()

# Make radio button less cramped by adding a space after each label
# The spacing will only show up in your IDE, not on this doc page
# options = list(eu_data['country_name'].unique())
# labels = [option + ' ' for option in options]

# input_dropdown = alt.binding_select(
#     # Add the empty selection which shows all when clicked
#     options=options,
#     labels=labels,
#     name='Region: '
# )

# countryselection = alt.selection_point(
#     fields=['country_name'],
#     bind=input_dropdown,
# )

slider2 = alt.binding_range(min=eu_data['year'].min(), 
                            max=eu_data['year'].max(), step=4, name='Current Year: ')

yearselector = alt.selection_point(
    name="YearSelector",
    fields=['year'],
    bind=slider2,
    value=[{'year': 1999}]
)

# partyselect = alt.selection_point(fields=['party_abbreviation'], bind='legend')

base = alt.Chart(eu_data).mark_circle().encode(
    x=alt.X('v2paminor:Q', scale=alt.Scale(domain=(-5, 5))),
    y=alt.Y('v2paimmig:Q', scale=alt.Scale(domain=(-5, 5))),
    size = 'v2paseatpercentage:Q',
    color = 'country_name:N',
    tooltip = ['v2paenname', 'country_name']
    
    # We need to set a constant domain to preserve the colors
    # when only one region is shown at a time
    # tooltip=['country','regionname','party_abbreviation'],
    # size=alt.Size('partyvote'),
    #opacity=alt.condition(partyselect, alt.value(1), alt.value(0.2)),
    #.scale(domain=options),
).add_params(
    yearselector
).transform_filter(
    yearselector
)


base.display()

In [ ]:
world = data.world_110m.url
world

In [ ]:
missing_values_count = world_data.isnull().sum()

print(missing_values_count['year'])

# print(missing_values_count['v2paminor'])

In [ ]:
# Columns with no missing data
world_data.columns[world_data.isnull().any() == False]

In [ ]:
test = eu_data.dropna(subset=['v2paimmig', 'v2paminor', 'year', 'country_name'])
print(test.shape)

In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type= 'mercator',
    scale= 350,                          # Magnify
    center= [20,50],                     # [lon, lat]
    clipExtent= [[0, 0], [400, 300]],
).properties(
    width=400, 
    height=300,
)

In [ ]:
import altair as alt
from vega_datasets import data

sphere = alt.sphere()
graticule = alt.graticule(step=[10, 10])
lats = alt.sequence(start=-30, stop=71, step=10, as_='lats')
lons = alt.sequence(start=-90, stop=91, step=10, as_='lons')

width = 600
height = 450

# Source of land data
source = alt.topo_feature(data.world_110m.url, 'countries')

# Layering and configuring the components 
base = alt.layer(
    alt.Chart(sphere).mark_geoshape(fill='none'), 
    alt.Chart(graticule).mark_geoshape(stroke='gray', strokeWidth=0.5), 
    alt.Chart(source).mark_geoshape(fill='lightgray', stroke='gray')    
).properties(width=width, height=height)

projections = {
#     "Bonne": {
#         "type": "bonne",
#         "center": [0, 52],
#         "rotate": [-20, 0],
#         "parallel": 52,
#         "translate": [width/2, height/2],
#         "scale": 700,
#         "precision": 0.1
#     },
    "Equidistanc conic": {
        "type": "conicEquidistant",
        "center": [0, 52],
        "rotate": [-20, 0],        
#         "parallels": [35, 65],
        "translate": [width/2, height/2],
        "scale": 700,
        "precision": 0.1        
    },
    "Lambert conformal conic": {
        "type": "conicConformal",
        "center": [0, 52],
        "rotate": [-20, 0],
#         "parallels": [35, 65],
        "translate": [width/2, height/2],
        "scale": 700,
        "precision": 0.1        
    },
    "Albers": {
        "type": "albers",
        "center": [0, 52],
        "rotate": [-20, 0],
#         "parallels": [35, 65],
        "translate": [width/2, height/2],
        "scale": 700,
        "precision": 0.1        
    },
    "Lambert azimuthal equal-area projection": {
        "type": "azimuthalEqualArea",        
        "rotate": [-20, -52],
        "translate": [width/2, height/2],
        "scale": 700,
        "precision": 0.1        
    }    
}

charts = [base.properties(projection=projections[key], title=key) for key in projections.keys()]
alt.concat(*charts, columns=2)

In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='albers',
    center=[0,52],
    rotate=[-20,0],
    translate=[300, 225],
    scale=700,
    precision=0.1
).properties(
    width=600, 
    height=450,
)

## Notes:

maybe use all countries in europe, not just EU

In [ ]:
import altair as alt
import pandas as pd

# Assuming df is your DataFrame and it has columns 'age' and 'height'
df = pd.DataFrame({
    'student': ['A', 'B', 'C', 'D', 'E'],
    'age': [15, 16, 15, 17, 16],
    'height': [160, 165, 170, 175, 180]
})

# Create a dropdown selection
selection = alt.selection_single(
    name='Select',
    fields=['option'],
    #init={'option': 'age'},
    bind=alt.binding_select(options=['age', 'height'])
)

# Create a chart
chart = alt.Chart(df).mark_bar().encode(
    x='student:N',
    y='value:Q',
    color=alt.Color('value:Q', legend=alt.Legend(title="Species by color")),
    tooltip=['student:N', 'value:Q']
).transform_fold(
    ['age', 'height'],
    as_=['option', 'value']
).add_selection(
    selection
).transform_filter(
    selection
)

chart


In [ ]:
import altair as alt
from vega_datasets import data

# Load the cars dataset
cars = data.cars()

# Calculate the quartiles of horsepower
q1 = cars['Horsepower'].quantile(0.25)
median = cars['Horsepower'].quantile(0.5)
q3 = cars['Horsepower'].quantile(0.75)
max_hp = cars['Horsepower'].max()

# Create a new column for the horsepower category
cars['Horsepower Category'] = '0 to Q1'
cars.loc[(cars['Horsepower'] > q1) & (cars['Horsepower'] <= median), 'Horsepower Category'] = 'Q1 to Median'
cars.loc[(cars['Horsepower'] > median) & (cars['Horsepower'] <= q3), 'Horsepower Category'] = 'Median to Q3'
cars.loc[cars['Horsepower'] > q3, 'Horsepower Category'] = 'Q3 to Max'

# Create a dropdown selection
selection = alt.selection_single(fields=['Horsepower Category'], bind=alt.binding_select(options=['0 to Q1', 'Q1 to Median', 'Median to Q3', 'Q3 to Max']))

# Create a scatterplot
scatter = alt.Chart(cars).mark_circle().encode(
    x='Year:T',
    y='Origin:N',
    size=alt.Size('sum(Miles_per_Gallon):Q', title='Total Miles per Gallon'),
    color='Origin:N',
    tooltip=['Year', 'Origin', 'sum(Miles_per_Gallon)']
).transform_filter(
    selection
).properties(
    title='Cars Scatterplot'
)

# Add the selection to the chart
chart = scatter.add_selection(
    selection
)

chart


In [ ]:
import pandas as pd

# Create a DataFrame
data = {
    'Area': ['BE10', 'BE10', 'BE10'],
    'Year': [2005, 2005, 2006],
    'Party': ['FPO', 'SPP', 'FPO'],
    'Partyvote': [24, 21, 28],
    'totalvote': [50, 50, 90],
    'voteshare': [0.48, 0.42, 0.31],
    'v2anti': [0.6, 0.03, 0.9]
}
df = pd.DataFrame(data)

import altair as alt

# Create a new column for the v2anti category
df['v2anti Category'] = '0 to 0.5'
df.loc[df['v2anti'] > 0.5, 'v2anti Category'] = '0.5 to 1'

# Create a dropdown selection
selection = alt.selection_single(fields=['v2anti Category'], bind=alt.binding_select(options=['0 to 0.5', '0.5 to 1']))

# Create a chart
chart = alt.Chart(df).mark_bar().encode(
    x='Area:N',
    y='sum(voteshare):Q',
    color='Year:N',
    column='Year:N'
).transform_filter(
    selection
).properties(
    title='Sum of Voteshare by Area and Year'
)

# Add the selection to the chart
chart = chart.add_selection(
    selection
)

chart
